# Interactive Visualization of the Volatility Decay

Based on a [reddit](https://www.reddit.com/r/HFEA/comments/tue7n6/the_volatility_decay_equation_with_verification/) post, make an interactive visualization to show the effect of the volatility decay.

The results show the (somewhat) quadratic (/ logarithmic) volatility drag along the volatility axis, together with (somewhat) quadratic (/ logarithmic) scaling profit region decrease with increased leverage factor. Further sources describing the quadratic behaviour:
- [Blogpost](https://www.afrugaldoctor.com/home/leveraged-etfs-and-volatility-decay-part-2)
- [(Detailed) Journal Article, also mentioned in the Blogpost](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1664823)

In [ ]:
import numpy as np

def leveraged_return(lev_factor: float, cagr_underlying: float, leverage_expense_ratio: float, \
                     libor: float, yearly_volatility: float) -> float:
    """
    Calculate the leveraged return according to 
    https://www.reddit.com/r/HFEA/comments/tue7n6/the_volatility_decay_equation_with_verification/

    :param lev_factor: float, leverage factor applied
    :param cagr_underlying: float, compound annual growth rate
    :param leverage_expense_ratio: float, expense ratio of the leveraged position (fund)
    :param libor: float, average LIBOR during investment period + 0.4%
    :param yearly_volatility: float, annualized volatility
    :return: float, annual return of leveraged position (fund)
    """
    # short names/ notation
    x = lev_factor
    r = cagr_underlying
    E = leverage_expense_ratio
    I = libor 
    s = yearly_volatility / np.sqrt(252)  # get daily volatility

    # define helpful quantities to avoid repitition & better overview
    exp = np.exp(np.log(1+r) / 252)
    e_i = (E + 1.1*(x-1)*I)/252
    first = x*s + x*s**2/(2*exp) + x*exp - e_i - x + 1
    second = x*exp**2 / (s + 0.5*s**2*exp**(-1) + exp) - e_i - x + 1

    return  (first*second)**126 - 1

# example calculation of leveraged return in percent (* 100)
leveraged_return(2, 0.4, 0.006, 0.005, 0.) * 100

In [ ]:
import ipywidgets as widgets
import plotly.graph_objects as go 

from ipywidgets import *

# FUTURE: For export possibilities, check out
# https://community.plotly.com/t/export-plotly-and-ipywidgets-as-an-html-file/18579
# and specifically the voila python project


def leveraged_return_mesh(lev: float, cagr_undr: float, exp: float, \
                          lib: float, vol_undr: float) -> np.ndarray:
    """
    Create a mesh of leveraged returns for visualizing the leveraged return against
    the underlying return and against the volatility. Return shows quadratic behaviour
    similar to the some discussion in the following sources:
    - https://www.afrugaldoctor.com/home/leveraged-etfs-and-volatility-decay-part-2
    - https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1664823

    :param lev: float, leverage factor applied
    :param cagr_undr: float, compound annual growth rate in percent
    :param exp: float, expense ratio of the leveraged position (fund) in percent
    :param lib: float, average LIBOR during investment period in percent
    :param vol_undr: float, annualized volatility of the underlying in percent
    :return: np.ndarray, volatility & underlying CAGR leveraged return mesh
    """
    # create mesh leveraged CAGR as array
    mesh = np.zeros((len(vol_undr), len(cagr_undr)))

    for i, vol in enumerate(vol_undr):
        for j, cagr in enumerate(cagr_undr):
            # reflect on volatility axis due to the way, plotly sets-up heatmaps
            # also, rescale percentage values, as otherwise not readable in the sliders
            mesh[i, j] = leveraged_return(lev, cagr / 100, exp / 100, \
                                          lib / 100, vol / 100) - cagr / 100

    return mesh * 100


# define parameters (except for leverage all in percent)
lev_r = 2
exp_r = 0.6
libor = 0.5


# define heatmap marginals (-50% - 50% underlying CAGR, 0-100% annualized volatility)
cagr_underlying = np.linspace(-50, 50, 200)
volatility_undr = np.linspace(0., 100, 100)

data = [go.Heatmap(x=cagr_underlying, y=volatility_undr, 
                   z=leveraged_return_mesh(lev_r, cagr_underlying, exp_r, libor, volatility_undr),
                   zmax=20, zmid=0, zmin=-15, colorscale="RdBu")]


fig = go.FigureWidget(data=data)

fig.update_layout(
    title="Visualized Gain over Unleveraged ETF",
    xaxis_title="CAGR Underlying [%]",
    yaxis_title="Volatility [%]",
)


@interact(leverage=(0, 10, 0.5), TER=(0, 1, 0.05), LIBOR=(0, 3, 0.25))
def update(leverage=lev_r, TER=exp_r, LIBOR=libor):
    """
    Interactive plot parameter updates.
    """
    with fig.batch_update():
        fig.data[0].z = leveraged_return_mesh(leverage, cagr_underlying, TER, LIBOR, volatility_undr)
    return fig

In [ ]:
# Detailed plot in much narrower region

# define heatmap marginals (-15% - 15% underlying CAGR, 0-40% annualized volatility)
cagr_underlying = np.linspace(-15, 15, 200)
volatility_undr = np.linspace(0., 40, 100)

data = [go.Contour(x=cagr_underlying, y=volatility_undr, 
                   z=leveraged_return_mesh(lev_r, cagr_underlying, exp_r, libor, volatility_undr),
                   zmax=20, zmid=0, zmin=-15, colorscale="RdBu")]


fig = go.FigureWidget(data=data)

fig.update_layout(
    title="Visualized Gain over Unleveraged ETF",
    xaxis_title="CAGR Underlying [%]",
    yaxis_title="Volatility [%]",
)


@interact(leverage=(0, 10, 0.5), TER=(0, 1, 0.05), LIBOR=(0, 3, 0.25))
def update(leverage=lev_r, TER=exp_r, LIBOR=libor):
    """
    Interactive plot parameter updates.
    """
    with fig.batch_update():
        fig.data[0].z = leveraged_return_mesh(leverage, cagr_underlying, TER, LIBOR, volatility_undr)
    return fig